In [1]:
import numpy as np
from pathlib import Path
import re
import pandas as pd
import networkx as nx
from cloudvolume import CloudVolume, Skeleton
from io import StringIO
import os
from brainlit.utils.util import (
    check_type,
    check_size,
)
from brainlit.utils.Neuron_trace import NeuronTrace

/Users/alishakodibagkar/opt/anaconda3/envs/brainlit/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


# NeuronTrace Class

The NeuronTrace class takes neuron data either stored in .swc format or located in an s3 bucket, and loads it as a dataframe.

If the neuron is stored in a .swc file, the swc filepath must be provided, and if the neuron is stored in an s3 bucket, the url, segment ID, and mip (resolution to use for scaling) must be provided.

In [2]:
swc_path = "../../../../../tree_2.swc"
s3_path = "s3://open-neurodata/brainlit/brain1_segments"
seg_id = 11
mip = 2

swc_trace = NeuronTrace(swc_path)
s3_trace = NeuronTrace(s3_path,seg_id,mip)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 15.36it/s]


## Functions of NeuronTrace Class

### 1. get_df_arguments

This function returns the arguments of the dataframe - for .swc files these will include offset, color, cc, branch, and for s3 files there should be none


In [3]:
swc_trace.get_df_arguments()

([73954.8686, 17489.532566, 34340.365689], [1.0, 1.0, 1.0], nan, nan)

In [4]:
s3_trace.get_df_arguments()

### 2. get_df

This function returns the dataframe object for the input neuron

In [5]:
swc_trace.get_df()

,sample,structure,x,y,z,r,parent
0,1,0,-52.589700,-1.448032,-1.228827,1.0,-1
1,2,0,-52.290940,-1.448032,-1.228827,1.0,1
2,3,0,-51.992181,-1.143616,-0.240423,1.0,2
3,4,0,-51.095903,-1.143616,-0.240423,1.0,3
4,5,0,-50.797144,-0.839201,-0.240423,1.0,4
...,...,...,...,...,...,...,...
148,149,0,45.702088,14.381594,-7.159252,1.0,148
149,150,0,46.000847,14.686010,-7.159252,1.0,149
150,151,0,46.897125,14.686010,-7.159252,1.0,150
151,152,0,47.494643,15.294842,-7.159252,1.0,151


In [6]:
s3_trace.get_df()

,sample,structure,x,y,z,r,parent
0,1,0,1104.0,1718.0,1396.0,1.0,-1
1,2,0,1120.0,1703.0,1399.0,1.0,1
2,3,0,1134.0,1691.0,1400.0,1.0,2
3,4,0,1152.0,1675.0,1403.0,1.0,3
4,14,0,1172.0,1655.0,1407.0,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,1308.0,1477.0,1415.0,1.0,354
2559,255,192,1296.0,1476.0,1417.0,1.0,317
2560,194,64,1283.0,1465.0,1420.0,1.0,255
2561,167,0,1275.0,1455.0,1423.0,1.0,194


### 3. get_skel

This function returns the Skeleton object of the dataframe, if the input file is a swc

Note: Currently, there is a problem when running get_skel on swc input traces. Appears to be an issue with code provided in _swc2skeleton function

In [7]:
swc_trace.get_skel()

ValueError: invalid literal for int() with base 10: './.'

In [8]:
s3_trace.get_skel()

Downloading: 100%|██████████| 1/1 [00:00<00:00,  9.62it/s]


Skeleton(segid=11, vertices=(shape=2563, float32), edges=(shape=2562, uint32), radius=(2563, float32), vertex_types=(2563, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])

### 4. get_df_voxel

This function provides the dataframe converted from spatial units to voxel, if spacing and an origin is provided. If origin is not specified, it defaults to (0,0,0)

In [9]:
#swc trace - origin not provided
swc_trace.get_df_voxel(spacing=np.asarray([2,2,2]))

,sample,structure,x,y,z,r,parent
0,1,0,-26,-1,-1,1.0,-1
1,2,0,-26,-1,-1,1.0,1
2,3,0,-26,-1,0,1.0,2
3,4,0,-26,-1,0,1.0,3
4,5,0,-25,0,0,1.0,4
...,...,...,...,...,...,...,...
148,149,0,23,7,-4,1.0,148
149,150,0,23,7,-4,1.0,149
150,151,0,23,7,-4,1.0,150
151,152,0,24,8,-4,1.0,151


In [10]:
#swc trace - origin provided
swc_trace.get_df_voxel(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

,sample,structure,x,y,z,r,parent
0,1,0,-276,-251,-251,1.0,-1
1,2,0,-276,-251,-251,1.0,1
2,3,0,-276,-251,-250,1.0,2
3,4,0,-276,-251,-250,1.0,3
4,5,0,-275,-250,-250,1.0,4
...,...,...,...,...,...,...,...
148,149,0,-227,-243,-254,1.0,148
149,150,0,-227,-243,-254,1.0,149
150,151,0,-227,-243,-254,1.0,150
151,152,0,-226,-242,-254,1.0,151


In [11]:
#s3 trace - origin not provided
s3_trace.get_df_voxel(spacing=np.asarray([2,2,2]))

,sample,structure,x,y,z,r,parent
0,1,0,552,859,698,1.0,-1
1,2,0,560,852,700,1.0,1
2,3,0,567,846,700,1.0,2
3,4,0,576,838,702,1.0,3
4,14,0,586,828,704,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,654,738,708,1.0,354
2559,255,192,648,738,708,1.0,317
2560,194,64,642,732,710,1.0,255
2561,167,0,638,728,712,1.0,194


In [12]:
#s3 trace - origin provided
s3_trace.get_df_voxel(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

,sample,structure,x,y,z,r,parent
0,1,0,302,609,448,1.0,-1
1,2,0,310,602,450,1.0,1
2,3,0,317,596,450,1.0,2
3,4,0,326,588,452,1.0,3
4,14,0,336,578,454,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,404,488,458,1.0,354
2559,255,192,398,488,458,1.0,317
2560,194,64,392,482,460,1.0,255
2561,167,0,388,478,462,1.0,194


### 5. get_graph

This function provides the dataframe in graph format. 
If spacing and origin is specified, the units will be converted from spatial to voxel units prior to making the graph.
If only spacing is specified, origin will be set to (0,0,0)


In [13]:
#swc input
swc_trace.get_graph()

In [14]:
#swc input, only spacing specified, origin defaults to (0,0,0)
swc_trace.get_graph(spacing=np.asarray([2,2,2]))

In [15]:
#swc input, spacing and origin specified
swc_trace.get_graph(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

In [16]:
#s3 input
s3_trace.get_graph()

In [17]:
#s3 input, only spacing specified, origin defaults to (0,0,0)
s3_trace.get_graph(spacing=np.asarray([2,2,2]))

In [18]:
#s3 input, spacing and origin specified
s3_trace.get_graph(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

### 6. get_paths

This function provides the dataframe as a list of paths. If spacing and origin is specified, the units will be converted from spatial to voxel units prior to making the paths. If only spacing is specified, origin will be set to (0,0,0)

In [19]:
#swc input
swc_trace.get_paths()

array([[[-52, -1, -1],
        [-52, -1, -1],
        [-51, -1, 0],
        [-51, -1, 0],
        [-50, 0, 0],
        [-50, 0, 0],
        [-49, 0, 0],
        [-48, 0, 0],
        [-46, 0, 0],
        [-46, 0, 0],
        [-45, 0, 0],
        [-45, -1, 0],
        [-41, -1, 0],
        [-41, -1, 0],
        [-40, -1, 0],
        [-39, -1, 0],
        [-39, -1, -1],
        [-38, -1, -1],
        [-37, -2, -1],
        [-37, -2, -1],
        [-37, -2, -1],
        [-36, -2, -1],
        [-36, -2, -1],
        [-35, -2, -1],
        [-31, -2, -1],
        [-31, -2, 0],
        [-29, -2, 0],
        [-29, -2, 0],
        [-28, -2, 0],
        [-28, -3, 0],
        [-26, -3, 0],
        [-26, -3, 0],
        [-25, -3, 0],
        [-24, -4, 0],
        [-24, -4, 0],
        [-23, -4, 0],
        [-23, -4, 0],
        [-19, -4, 0],
        [-18, -3, 0],
        [-17, -3, 0],
        [-16, -3, 0],
        [-16, -3, 0],
        [-16, -2, 0],
        [-15, -2, -1],
        [-15, -2, -2],
    

In [20]:
#swc input, only spacing specified, origin defaults to (0,0,0)
swc_trace.get_paths(spacing=np.asarray([2,2,2]))

array([[[-26, -1, -1],
        [-26, -1, -1],
        [-26, -1, 0],
        [-26, -1, 0],
        [-25, 0, 0],
        [-25, 0, 0],
        [-25, 0, 0],
        [-24, 0, 0],
        [-23, 0, 0],
        [-23, 0, 0],
        [-23, 0, 0],
        [-23, -1, 0],
        [-21, -1, 0],
        [-21, -1, 0],
        [-20, -1, 0],
        [-20, -1, 0],
        [-20, -1, -1],
        [-19, -1, -1],
        [-19, -1, -1],
        [-19, -1, -1],
        [-19, -1, -1],
        [-18, -1, -1],
        [-18, -1, -1],
        [-18, -1, -1],
        [-16, -1, -1],
        [-16, -1, 0],
        [-15, -1, 0],
        [-15, -1, 0],
        [-14, -1, 0],
        [-14, -2, 0],
        [-13, -2, 0],
        [-13, -2, 0],
        [-13, -2, 0],
        [-12, -2, 0],
        [-12, -2, 0],
        [-12, -2, 0],
        [-12, -2, 0],
        [-10, -2, 0],
        [-9, -2, 0],
        [-9, -2, 0],
        [-8, -2, 0],
        [-8, -2, 0],
        [-8, -1, 0],
        [-8, -1, -1],
        [-8, -1, -1],
        [-8

In [21]:
#swc input, spacing and origin specified
swc_trace.get_paths(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

array([[[-276, -251, -251],
        [-276, -251, -251],
        [-276, -251, -250],
        [-276, -251, -250],
        [-275, -250, -250],
        [-275, -250, -250],
        [-275, -250, -250],
        [-274, -250, -250],
        [-273, -250, -250],
        [-273, -250, -250],
        [-273, -250, -250],
        [-273, -251, -250],
        [-271, -251, -250],
        [-271, -251, -250],
        [-270, -251, -250],
        [-270, -251, -250],
        [-270, -251, -251],
        [-269, -251, -251],
        [-269, -251, -251],
        [-269, -251, -251],
        [-269, -251, -251],
        [-268, -251, -251],
        [-268, -251, -251],
        [-268, -251, -251],
        [-266, -251, -251],
        [-266, -251, -250],
        [-265, -251, -250],
        [-265, -251, -250],
        [-264, -251, -250],
        [-264, -252, -250],
        [-263, -252, -250],
        [-263, -252, -250],
        [-263, -252, -250],
        [-262, -252, -250],
        [-262, -252, -250],
        [-262, -252,

In [22]:
#s3 input
s3_trace.get_paths()

array([array([[1104, 1718, 1396],
       [1120, 1703, 1399],
       [1134, 1691, 1400],
       [1152, 1675, 1403],
       [1172, 1655, 1407],
       [1184, 1642, 1410],
       [1193, 1629, 1412],
       [1203, 1616, 1415],
       [1213, 1601, 1415],
       [1228, 1581, 1416],
       [1238, 1568, 1416],
       [1253, 1556, 1414],
       [1262, 1549, 1414],
       [1274, 1537, 1415],
       [1283, 1527, 1414],
       [1289, 1517, 1415],
       [1298, 1500, 1414],
       [1306, 1491, 1413],
       [1312, 1483, 1413],
       [1315, 1475, 1413],
       [1327, 1470, 1411],
       [1331, 1471, 1409],
       [1335, 1474, 1409],
       [1343, 1473, 1410],
       [1346, 1473, 1411],
       [1356, 1474, 1412],
       [1364, 1474, 1411],
       [1368, 1472, 1410],
       [1375, 1471, 1409],
       [1388, 1470, 1406],
       [1398, 1467, 1405],
       [1402, 1459, 1405],
       [1403, 1448, 1407],
       [1405, 1439, 1407],
       [1408, 1423, 1408],
       [1415, 1414, 1412],
       [1421, 1411, 1

In [23]:
#s3 input, only spacing specified, origin defaults to (0,0,0)
s3_trace.get_paths(spacing=np.asarray([2,2,2]))

array([array([[ 552,  859,  698],
       [ 560,  852,  700],
       [ 567,  846,  700],
       [ 576,  838,  702],
       [ 586,  828,  704],
       [ 592,  821,  705],
       [ 596,  814,  706],
       [ 602,  808,  708],
       [ 606,  800,  708],
       [ 614,  790,  708],
       [ 619,  784,  708],
       [ 626,  778,  707],
       [ 631,  774,  707],
       [ 637,  768,  708],
       [ 642,  764,  707],
       [ 644,  758,  708],
       [ 649,  750,  707],
       [ 653,  746,  706],
       [ 656,  742,  706],
       [ 658,  738,  706],
       [ 664,  735,  706],
       [ 666,  736,  704],
       [ 668,  737,  704],
       [ 672,  736,  705],
       [ 673,  736,  706],
       [ 678,  737,  706],
       [ 682,  737,  706],
       [ 684,  736,  705],
       [ 688,  736,  704],
       [ 694,  735,  703],
       [ 699,  734,  702],
       [ 701,  730,  702],
       [ 702,  724,  704],
       [ 702,  720,  704],
       [ 704,  712,  704],
       [ 708,  707,  706],
       [ 710,  706,  

In [24]:
#s3 input, spacing and origin specified
s3_trace.get_paths(spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

array([array([[ 302,  609,  448],
       [ 310,  602,  450],
       [ 317,  596,  450],
       [ 326,  588,  452],
       [ 336,  578,  454],
       [ 342,  571,  455],
       [ 346,  564,  456],
       [ 352,  558,  458],
       [ 356,  550,  458],
       [ 364,  540,  458],
       [ 369,  534,  458],
       [ 376,  528,  457],
       [ 381,  524,  457],
       [ 387,  518,  458],
       [ 392,  514,  457],
       [ 394,  508,  458],
       [ 399,  500,  457],
       [ 403,  496,  456],
       [ 406,  492,  456],
       [ 408,  488,  456],
       [ 414,  485,  456],
       [ 416,  486,  454],
       [ 418,  487,  454],
       [ 422,  486,  455],
       [ 423,  486,  456],
       [ 428,  487,  456],
       [ 432,  487,  456],
       [ 434,  486,  455],
       [ 438,  486,  454],
       [ 444,  485,  453],
       [ 449,  484,  452],
       [ 451,  480,  452],
       [ 452,  474,  454],
       [ 452,  470,  454],
       [ 454,  462,  454],
       [ 458,  457,  456],
       [ 460,  456,  

### 7. generate_df_subset

This function reads a new subset dataframe, taking in a list of voxels. An option was added to provide a subset of the neuron rather than the entire neuron (by providing subneuron_start and subneuron_end)

In [25]:
#swc input, no subneuron_start and subneuron_end

#generate vox_in_img_list
my_list = []
for i in range(len(swc_trace.get_df())):
    my_list.append(10)
vox_in_img_list = [my_list,my_list,my_list]

swc_trace.generate_df_subset(vox_in_img_list)

,sample,structure,x,y,z,r,parent
0,1,0,10,10,10,1.0,-1
1,2,0,10,10,10,1.0,1
2,3,0,10,10,10,1.0,2
3,4,0,10,10,10,1.0,3
4,5,0,10,10,10,1.0,4
...,...,...,...,...,...,...,...
148,149,0,10,10,10,1.0,148
149,150,0,10,10,10,1.0,149
150,151,0,10,10,10,1.0,150
151,152,0,10,10,10,1.0,151


In [26]:
#swc input, subneuron_start and subneuron_end specified

subneuron_start = 5
subneuron_end = 8

#generate vox_in_img_list
my_list = []
for i in range(subneuron_end-subneuron_start):
    my_list.append(10)
vox_in_img_list_2 = [my_list,my_list,my_list]

swc_trace.generate_df_subset(vox_in_img_list_2,subneuron_start,subneuron_end)

,sample,structure,x,y,z,r,parent
5,6,0,10,10,10,1.0,5
6,7,0,10,10,10,1.0,6
7,8,0,10,10,10,1.0,7


In [27]:
#s3 input, no subneuron_start and subneuron_end

#generate vox_in_img_list
my_list = []
for i in range(len(s3_trace.get_df())):
    my_list.append(10)
vox_in_img_list_3 = [my_list,my_list,my_list]

s3_trace.generate_df_subset(vox_in_img_list_3)

,sample,structure,x,y,z,r,parent
0,1,0,10,10,10,1.0,-1
1,2,0,10,10,10,1.0,1
2,3,0,10,10,10,1.0,2
3,4,0,10,10,10,1.0,3
4,14,0,10,10,10,1.0,4
...,...,...,...,...,...,...,...
2558,317,0,10,10,10,1.0,354
2559,255,192,10,10,10,1.0,317
2560,194,64,10,10,10,1.0,255
2561,167,0,10,10,10,1.0,194


In [28]:
#s3 input, subneuron_start and subneuron_end specified

subneuron_start = 5
subneuron_end = 8

#generate vox_in_img_list
my_list = []
for i in range(subneuron_end-subneuron_start):
    my_list.append(10)
vox_in_img_list_4 = [my_list,my_list,my_list]

s3_trace.generate_df_subset(vox_in_img_list_4,subneuron_start,subneuron_end)

,sample,structure,x,y,z,r,parent
5,29,0,10,10,10,1.0,14
6,44,0,10,10,10,1.0,29
7,57,0,10,10,10,1.0,44


### 8. get_bfs_subgraph

This function creates a spanning subgraph from a seed node and parent graph created from the dataframes using BFS. The seed node ID and max depth for BFS should be specified. A dataframe storing indices can be specified. If spacing and origin is specified, the units will be converted from spatial to voxel units. If only spacing is specified, origin will be set to (0,0,0).


In [29]:
#swc input, specify node_id and depth
swc_trace.get_bfs_subgraph(node_id=11,depth=2)

(<networkx.classes.digraph.DiGraph at 0x7fe47a53b970>,
 <networkx.classes.digraph.DiGraph at 0x7fe499cc9f40>)

In [30]:
#swc input, provide a dataframe
swc_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df())

(<networkx.classes.digraph.DiGraph at 0x7fe47a53b7f0>,
 <networkx.classes.digraph.DiGraph at 0x7fe499383b80>)

In [31]:
#swc input, add spacing 
swc_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]))

(<networkx.classes.digraph.DiGraph at 0x7fe499cc6af0>,
 <networkx.classes.digraph.DiGraph at 0x7fe4995e9a00>)

In [32]:
#swc input, add spacing and origin
swc_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]),origin=np.asarray([50,50,50]))

(<networkx.classes.digraph.DiGraph at 0x7fe47a7bce80>,
 <networkx.classes.digraph.DiGraph at 0x7fe49086c3d0>)

In [33]:
#s3 input, specify node_id and depth
s3_trace.get_bfs_subgraph(node_id=11,depth=2)

(<networkx.classes.digraph.DiGraph at 0x7fe47a7bc0a0>,
 <networkx.classes.digraph.DiGraph at 0x7fe4993b7700>)

In [34]:
#s3 input, provide a dataframe
s3_trace.get_bfs_subgraph(node_id=11,depth=2,df=swc_trace.get_df())

(<networkx.classes.digraph.DiGraph at 0x7fe499cc9490>,
 <networkx.classes.digraph.DiGraph at 0x7fe47a7bc160>)

In [35]:
#s3 input, add spacing 
s3_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]))

(<networkx.classes.digraph.DiGraph at 0x7fe47a575130>,
 <networkx.classes.digraph.DiGraph at 0x7fe47a53b340>)

In [36]:
#s3 input, add spacing and origin
s3_trace.get_bfs_subgraph(node_id=11,depth=2,df=s3_trace.get_df(),spacing=np.asarray([2,2,2]),origin=np.asarray([50,50,50]))

(<networkx.classes.digraph.DiGraph at 0x7fe47a575fa0>,
 <networkx.classes.digraph.DiGraph at 0x7fe47a53ba30>)

### 9. get_sub_neuron

This function returns a sub-neuron in graph format with node coordinates bounded by a bounding box with start and end. If spacing and origin is specified, the units will be converted from spatial to voxel units. If only spacing is specified, origin will be set to (0,0,0).

In [37]:
bounding_box=[[1,2,4],[1,2,3]]

#swc input, no spacing and origin
swc_trace.get_sub_neuron(bounding_box)

In [38]:
#swc input, spacing specified
swc_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]))

In [39]:
#swc input, spacing and origin specified
swc_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))

In [40]:
#s3 input, no spacing and origin
s3_trace.get_sub_neuron(bounding_box)

In [41]:
#s3 input, spacing specified
s3_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]))

In [42]:
#s3 input, spacing and origin specified
s3_trace.get_sub_neuron(bounding_box,spacing=np.asarray([2,2,2]),origin=np.asarray([500,500,500]))